### CoT_Expert

In [ ]:
import pandas as pd
from openai import OpenAI
from tqdm import tqdm
import time
from pathlib import Path
import os

EXCEL_FILE = "MedQA_gpt-oss-120b.xlsx"
SHEET_NAME = "Sheet1"
SAVE_EVERY = 5

API_KEY = os.getenv("NV_API_KEY", "")

client = OpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key=API_KEY
)

df = pd.read_excel(EXCEL_FILE, sheet_name=SHEET_NAME, engine="openpyxl")

if "Content_21" not in df.columns:
    df["Content_21"] = None

unprocessed_rows = df[df["Rationale_21"].isna() | (df["Rationale_21"] == "")]

if unprocessed_rows.empty:
    print("No unprocessed data.")
else:
    idx_list = list(unprocessed_rows.index)
    total = len(idx_list)

    for i, idx in enumerate(tqdm(idx_list, total=total, desc="processing"), start=1):
        row = df.loc[idx]

        user_input = "Provide a step-by-step deduction that identifies the correct response.\nExample Question 1:\nShortly after undergoing a bipolar prosthesis for a displaced femoral neck fracture of the left hip acquired after a fall the day before, an 80-year-old woman suddenly develops dyspnea. The surgery under general anesthesia with sevoflurane was uneventful, lasting 98 min, during which the patient maintained oxygen saturation readings of 100% on 8 l of oxygen. She has a history of hypertension, osteoporosis, and osteoarthritis of her right knee. Her medications include ramipril, naproxen, ranitidine, and a multivitamin. She appears cyanotic, drowsy, and is oriented only to person. Her temperature is 38.6 °C (101.5 °F), pulse is 135/min, respirations are 36/min, and blood pressure is 155/95 mm Hg. Pulse oximetry on room air shows an oxygen saturation of 81%. There are several scattered petechiae on the anterior chest wall. Laboratory studies show a hemoglobin concentration of 10.5 g/dl, a leukocyte count of 9000/mm3 , a platelet count of 145,000/mm3 , and a creatine kinase of 190 U/l. An ECG shows sinus tachycardia. What is the most likely diagnosis?\nExample Rationale 1:\nThe patient had a surgical repair of a displaced femoral neck fracture. The patient has petechiae. The patient has a new oxygen requirement, meaning they are having difficulty with their breathing. This patient most likely has a fat embolism. \nExample Question 2:\nA 55-year-old man comes to the emergency department because of a dry cough and severe chest pain beginning that morning. Two months ago, he was diagnosed with inferior wall myocardial infarction and was treated with stent implantation of the right coronary artery. He has a history of hypertension and hypercholesterolemia. His medications include aspirin, clopidogrel, atorvastatin, and enalapril. His temperature is 38.5°C (101.3 °F), pulse is 92/min, respirations are 22/min, and blood pressure is 130/80 mm Hg. Cardiac examination shows a high-pitched scratching sound best heard while sitting upright and during expiration. The remainder of the examination shows no abnormalities. An ECG shows diffuse ST elevations. Serum studies show a troponin I of 0.005 ng/ml (N < 0.01). What is the most likely cause of this patient's symptoms?\nExample Rationale 2:\nThis patient is having chest pain. They recently had a heart attack and has new chest pain, suggesting he may have a problem with his heart. The EKG has diffuse ST elevations and he has a scratching murmur. This patient likely has Dressler Syndrome.\n\nQuestion:\n" + str(row["question"])

        reasoning_text = ""
        final_content_text = ""

        try:
            completion = client.chat.completions.create(
                model="openai/gpt-oss-120b",
                messages=[{"role": "user", "content": user_input}],
                temperature=0,
                top_p=1,
                max_tokens=4096,
                stream=True,
                reasoning_effort="high"
            )

            for chunk in completion:
                try:
                    delta = chunk.choices[0].delta
                    reasoning = getattr(delta, "reasoning_content", None)
                    if reasoning is not None:
                        reasoning_text += reasoning
                    content = getattr(delta, "content", None)
                    if content is not None:
                        final_content_text += content
                except Exception:
                    continue

            df.at[idx, "Rationale_21"] = reasoning_text

            if not final_content_text and reasoning_text:
                df.at[idx, "Content_21"] = None
                print("content is empty.")
            else:
                df.at[idx, "Content_21"] = final_content_text

        except Exception as e:
            print(f"Error processing idx={idx}: {e}")
            error_message = f"[ERROR] {e}"
            df.at[idx, "Rationale_21"] = error_message
            df.at[idx, "Content_21"] = error_message

        if (i % SAVE_EVERY == 0) or (i == total):
            df.to_excel(EXCEL_FILE, index=False, engine="openpyxl")
            print(f"Saved progress after {i} / {total}")

    print("All unprocessed data has been processed.")


### DR_Doctor

In [ ]:
import pandas as pd
from openai import OpenAI
from tqdm import tqdm
import time
from pathlib import Path
import os

EXCEL_FILE = "MedQA_gpt-oss-120b.xlsx"
SHEET_NAME = "Sheet1"
SAVE_EVERY = 5

API_KEY = os.getenv("NV_API_KEY", "")

client = OpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key=API_KEY
)

df = pd.read_excel(EXCEL_FILE, sheet_name=SHEET_NAME, engine="openpyxl")

if "Content_21" not in df.columns:
    df["Content_21"] = None

unprocessed_rows = df[df["Rationale_21"].isna() | (df["Rationale_21"] == "")]

if unprocessed_rows.empty:
    print("No unprocessed data.")
else:
    idx_list = list(unprocessed_rows.index)
    total = len(idx_list)

    for i, idx in enumerate(tqdm(idx_list, total=total, desc="processing"), start=1):
        row = df.loc[idx]

        user_input = "Provide a step-by-step deduction that identifies the correct response.\nExample Question 1:\nShortly after undergoing a bipolar prosthesis for a displaced femoral neck fracture of the left hip acquired after a fall the day before, an 80-year-old woman suddenly develops dyspnea. The surgery under general anesthesia with sevoflurane was uneventful, lasting 98 min, during which the patient maintained oxygen saturation readings of 100% on 8 l of oxygen. She has a history of hypertension, osteoporosis, and osteoarthritis of her right knee. Her medications include ramipril, naproxen, ranitidine, and a multivitamin. She appears cyanotic, drowsy, and is oriented only to person. Her temperature is 38.6 °C (101.5 °F), pulse is 135/min, respirations are 36/min, and blood pressure is 155/95 mm Hg. Pulse oximetry on room air shows an oxygen saturation of 81%. There are several scattered petechiae on the anterior chest wall. Laboratory studies show a hemoglobin concentration of 10.5 g/dl, a leukocyte count of 9000/mm3 , a platelet count of 145,000/mm3 , and a creatine kinase of 190 U/l. An ECG shows sinus tachycardia. What is the most likely diagnosis?\nExample Rationale 1:\nThe patient had a surgical repair of a displaced femoral neck fracture. The patient has petechiae. The patient has a new oxygen requirement, meaning they are having difficulty with their breathing. This patient most likely has a fat embolism. \nExample Question 2:\nA 55-year-old man comes to the emergency department because of a dry cough and severe chest pain beginning that morning. Two months ago, he was diagnosed with inferior wall myocardial infarction and was treated with stent implantation of the right coronary artery. He has a history of hypertension and hypercholesterolemia. His medications include aspirin, clopidogrel, atorvastatin, and enalapril. His temperature is 38.5°C (101.3 °F), pulse is 92/min, respirations are 22/min, and blood pressure is 130/80 mm Hg. Cardiac examination shows a high-pitched scratching sound best heard while sitting upright and during expiration. The remainder of the examination shows no abnormalities. An ECG shows diffuse ST elevations. Serum studies show a troponin I of 0.005 ng/ml (N < 0.01). What is the most likely cause of this patient's symptoms?\nExample Rationale 2:\nThis patient is having chest pain. They recently had a heart attack and has new chest pain, suggesting he may have a problem with his heart. The EKG has diffuse ST elevations and he has a scratching murmur. This patient likely has Dressler Syndrome.\n\nQuestion:\n" + str(row["question"])

        reasoning_text = ""
        final_content_text = ""

        try:
            completion = client.chat.completions.create(
                model="openai/gpt-oss-120b",
                messages=[{"role": "user", "content": user_input}],
                temperature=0,
                top_p=1,
                max_tokens=4096,
                stream=True,
                reasoning_effort="high"
            )

            for chunk in completion:
                try:
                    delta = chunk.choices[0].delta
                    reasoning = getattr(delta, "reasoning_content", None)
                    if reasoning is not None:
                        reasoning_text += reasoning
                    content = getattr(delta, "content", None)
                    if content is not None:
                        final_content_text += content
                except Exception:
                    continue

            df.at[idx, "Rationale_21"] = reasoning_text

            if not final_content_text and reasoning_text:
                df.at[idx, "Content_21"] = None
                print("content is empty.")
            else:
                df.at[idx, "Content_21"] = final_content_text

        except Exception as e:
            print(f"Error processing idx={idx}: {e}")
            error_message = f"[ERROR] {e}"
            df.at[idx, "Rationale_21"] = error_message
            df.at[idx, "Content_21"] = error_message

        if (i % SAVE_EVERY == 0) or (i == total):
            df.to_excel(EXCEL_FILE, index=False, engine="openpyxl")
            print(f"Saved progress after {i} / {total}")

    print("All unprocessed data has been processed.")


### IR_Doctor

In [ ]:
import pandas as pd
from openai import OpenAI
from tqdm import tqdm
import time
from pathlib import Path
import os

EXCEL_FILE = "MedQA_gpt-oss-120b.xlsx"
SHEET_NAME = "Sheet1"
SAVE_EVERY = 5

API_KEY = os.getenv("NV_API_KEY", "")

client = OpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key=API_KEY
)

df = pd.read_excel(EXCEL_FILE, sheet_name=SHEET_NAME, engine="openpyxl")

if "Content_25" not in df.columns:
    df["Content_25"] = None

unprocessed_rows = df[df["Rationale_25"].isna() | (df["Rationale_25"] == "")]

if unprocessed_rows.empty:
    print("No unprocessed data.")
else:
    idx_list = list(unprocessed_rows.index)
    total = len(idx_list)

    for i, idx in enumerate(tqdm(idx_list, total=total, desc="processing"), start=1):
        row = df.loc[idx]

        user_input = "Use symptom, signs, and laboratory disease associations to step by step deduce the correct response.\nExample Question 1:\nShortly after undergoing a bipolar prosthesis for a displaced femoral neck fracture of the left hip acquired after a fall the day before, an 80-year-old woman suddenly develops dyspnea. The surgery under general anesthesia with sevoflurane was uneventful, lasting 98 min, during which the patient maintained oxygen saturation readings of 100% on 8 l of oxygen. She has a history of hypertension, osteoporosis, and osteoarthritis of her right knee. Her medications include ramipril, naproxen, ranitidine, and a multivitamin. She appears cyanotic, drowsy, and is oriented only to person. Her temperature is 38.6 °C (101.5 °F), pulse is 135/min, respirations are 36/min, and blood pressure is 155/95 mm Hg. Pulse oximetry on room air shows an oxygen saturation of 81%. There are several scattered petechiae on the anterior chest wall. Laboratory studies show a hemoglobin concentration of 10.5 g/dl, a leukocyte count of 9000/mm3 , a platelet count of 145,000/mm3 , and a creatine kinase of 190 U/l. An ECG shows sinus tachycardia. What is the most likely diagnosis?\nExample Rationale 1:\n This patient has findings of petechiae, altered mental status, shortness of breath, and recent surgery suggesting a diagnosis of fat emboli. The patient most likely has a fat embolism.\nExample Question 2:\nA 55-year-old man comes to the emergency department because of a dry cough and severe chest pain beginning that morning. Two months ago, he was diagnosed with inferior wall myocardial infarction and was treated with stent implantation of the right coronary artery. He has a history of hypertension and hypercholesterolemia. His medications include aspirin, clopidogrel, atorvastatin, and enalapril. His temperature is 38.5Â°C (101.3 °F), pulse is 92/min, respirations are 22/min, and blood pressure is 130/80 mm Hg. Cardiac examination shows a high-pitched scratching sound best heard while sitting upright and during expiration. The remainder of the examination shows no abnormalities. An ECG shows diffuse ST elevations. Serum studies show a troponin I of 0.005 ng/ml (N < 0.01). What is the most likely cause of this patient’s symptoms?\nExample Rationale 2:\n This patient had a recent myocardial infarction with new development of diffuse ST elevations, chest pain, and a high pitched scratching murmur which are found in Dressler’s syndrome. This patient likely has Dressler's Syndrome.\n\nQuestion:\n" + str(row["question"])


        reasoning_text = ""
        final_content_text = ""

        try:
            completion = client.chat.completions.create(
                model="openai/gpt-oss-120b",
                messages=[{"role": "user", "content": user_input}],
                temperature=0,
                top_p=1,
                max_tokens=4096,
                stream=True,
                reasoning_effort="high"
            )

            for chunk in completion:
                try:
                    delta = chunk.choices[0].delta

                    reasoning = getattr(delta, "reasoning_content", None)
                    if reasoning is not None:
                        reasoning_text += reasoning

                    content = getattr(delta, "content", None)
                    if content is not None:
                        final_content_text += content

                except Exception:
                    continue

            df.at[idx, "Rationale_25"] = reasoning_text

            if not final_content_text and reasoning_text:
                df.at[idx, "Content_25"] = None
                print("content is empty.")
            else:
                df.at[idx, "Content_25"] = final_content_text

        except Exception as e:
            print(f"Error processing idx={idx}: {e}")
            error_message = f"[ERROR] {e}"
            df.at[idx, "Rationale_25"] = error_message
            df.at[idx, "Content_25"] = error_message

        if (i % SAVE_EVERY == 0) or (i == total):
            df.to_excel(EXCEL_FILE, index=False, engine="openpyxl")
            print(f"Saved progress after {i} / {total}")

    print("All unprocessed data has been processed.")


### AR_Doctor

In [ ]:
import pandas as pd
from openai import OpenAI
from tqdm import tqdm
import time
from pathlib import Path
import os

EXCEL_FILE = "MedQA_gpt-oss-120b.xlsx"
SHEET_NAME = "Sheet1"
SAVE_EVERY = 5

API_KEY = os.getenv("NV_API_KEY", "")

client = OpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key=API_KEY
)

df = pd.read_excel(EXCEL_FILE, sheet_name=SHEET_NAME, engine="openpyxl")

if "Content_27" not in df.columns:
    df["Content_27"] = None

unprocessed_rows = df[df["Rationale_27"].isna() | (df["Rationale_27"] == "")]

if unprocessed_rows.empty:
    print("No unprocessed rows found.")
else:
    idx_list = list(unprocessed_rows.index)
    total = len(idx_list)

    for i, idx in enumerate(tqdm(idx_list, total=total, desc="processing"), start=1):
        row = df.loc[idx]

        user_input = "Use analytic reasoning to deduce the physiologic or biochemical pathophysiology of the patient and step by step identify the correct response.\nExample Question 1:\nShortly after undergoing a bipolar prosthesis for a displaced femoral neck fracture of the left hip acquired after a fall the day before, an 80-year-old woman suddenly develops dyspnea. The surgery under general anesthesia with sevoflurane was uneventful, lasting 98 min, during which the patient maintained oxygen saturation readings of 100% on 8 l of oxygen. She has a history of hypertension, osteoporosis, and osteoarthritis of her right knee. Her medications include ramipril, naproxen, ranitidine, and a multivitamin. She appears cyanotic, drowsy, and is oriented only to person. Her temperature is 38.6 °C (101.5 °F), pulse is 135/min, respirations are 36/min, and blood pressure is 155/95 mm Hg. Pulse oximetry on room air shows an oxygen saturation of 81%. There are several scattered petechiae on the anterior chest wall. Laboratory studies show a hemoglobin concentration of 10.5 g/dl, a leukocyte count of 9000/mm3 , a platelet count of 145,000/mm3 , and a creatine kinase of 190 U/l. An ECG shows sinus tachycardia. What is the most likely diagnosis?\nExample Rationale 1:\n The patient recently had large bone surgery making fat emboli a potential cause because the bone marrow was manipulated. Petechiae can form in response to capillary inflammation caused by fat emboli. Fat micro globules cause CNS microcirculation occlusion causing confusion and altered mental status. Fat obstruction in the pulmonary arteries can cause tachycardia and shortness of breath as seen in this patient. This patient most likely has a fat embolism.\nExample Question 2:\nA 55-year-old man comes to the emergency department because of a dry cough and severe chest pain beginning that morning. Two months ago, he was diagnosed with inferior wall myocardial infarction and was treated with stent implantation of the right coronary artery. He has a history of hypertension and hypercholesterolemia. His medications include aspirin, clopidogrel, atorvastatin, and enalapril. His temperature is 38.5Â°C (101.3 °F), pulse is 92/min, respirations are 22/min, and blood pressure is 130/80 mm Hg. Cardiac examination shows a high-pitched scratching sound best heard while sitting upright and during expiration. The remainder of the examination shows no abnormalities. An ECG shows diffuse ST elevations. Serum studies show a troponin I of 0.005 ng/ml (N < 0.01). What is the most likely cause of this patient’s symptoms?\nExample Rationale 2:\n This patient had a recent myocardial infarction which can cause myocardial inflammation that causes pericarditis and Dressler Syndrome. The diffuse ST elevations and high pitched scratching murmur can be signs of pericardial inflammation as the inflamed pericardium rubs against the pleura as seen with Dressler Syndrome. This patient likely has Dressler Syndrome.\n\nQuestion:\n" + str(row["question"])

        reasoning_text = ""
        final_content_text = ""

        try:
            completion = client.chat.completions.create(
                model="openai/gpt-oss-120b",
                messages=[{"role": "user", "content": user_input}],
                temperature=0,
                top_p=1,
                max_tokens=4096,
                stream=True,
                reasoning_effort="high"
            )

            for chunk in completion:
                try:
                    delta = chunk.choices[0].delta

                    reasoning = getattr(delta, "reasoning_content", None)
                    if reasoning is not None:
                        reasoning_text += reasoning

                    content = getattr(delta, "content", None)
                    if content is not None:
                        final_content_text += content

                except Exception:
                    continue

            df.at[idx, "Rationale_27"] = reasoning_text

            if not final_content_text and reasoning_text:
                df.at[idx, "Content_27"] = None
                print("content is empty.")
            else:
                df.at[idx, "Content_27"] = final_content_text

        except Exception as e:
            print(f"Error processing idx={idx}: {e}")
            error_message = f"[ERROR] {e}"
            df.at[idx, "Rationale_27"] = error_message
            df.at[idx, "Content_27"] = error_message

        if (i % SAVE_EVERY == 0) or (i == total):
            df.to_excel(EXCEL_FILE, index=False, engine="openpyxl")
            print(f"Saved progress after {i} / {total}")

    print("All unprocessed rows have been completed.")


### BR_Doctor

In [ ]:
import pandas as pd
from openai import OpenAI
from tqdm import tqdm
import time
from pathlib import Path
import os

EXCEL_FILE = "MedQA_gpt-oss-120b.xlsx"
SHEET_NAME = "Sheet1"
SAVE_EVERY = 5

API_KEY = os.getenv("NV_API_KEY", "")

client = OpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key=API_KEY
)

df = pd.read_excel(EXCEL_FILE, sheet_name=SHEET_NAME, engine="openpyxl")

if "Content_29" not in df.columns:
    df["Content_29"] = None

unprocessed_rows = df[df["Rationale_29"].isna() | (df["Rationale_29"] == "")]

if unprocessed_rows.empty:
    print("No unprocessed rows found.")
else:
    idx_list = list(unprocessed_rows.index)
    total = len(idx_list)

    for i, idx in enumerate(tqdm(idx_list, total=total, desc="processing"), start=1):
        row = df.loc[idx]

        user_input = "Use step-by-step Bayesian Inference to create a prior probability that is updated with new information in the history to produce a posterior probability and determine the final diagnosis.\nExample Question 1:\nShortly after undergoing a bipolar prosthesis for a displaced femoral neck fracture of the left hip acquired after a fall the day before, an 80-year-old woman suddenly develops dyspnea. The surgery under general anesthesia with sevoflurane was uneventful, lasting 98 min, during which the patient maintained oxygen saturation readings of 100% on 8 l of oxygen. She has a history of hypertension, osteoporosis, and osteoarthritis of her right knee. Her medications include ramipril, naproxen, ranitidine, and a multivitamin. She appears cyanotic, drowsy, and is oriented only to person. Her temperature is 38.6 °C (101.5 °F), pulse is 135/min, respirations are 36/min, and blood pressure is 155/95 mm Hg. Pulse oximetry on room air shows an oxygen saturation of 81%. There are several scattered petechiae on the anterior chest wall. Laboratory studies show a hemoglobin concentration of 10.5 g/dl, a leukocyte count of 9000/mm3 , a platelet count of 145,000/mm3 , and a creatine kinase of 190 U/l. An ECG shows sinus tachycardia. What is the most likely diagnosis?\nExample Rationale 1:\n The prior probability of fat embolism is 0.05% however the patient has petechiae on exam which is seen with fat emboli, which increases the posterior probability of fat embolism to 5%. Altered mental status increases the probability further to 10%. Recent orthopedic surgery increases the probability of fat emboli syndrome to 60%. This patient most likely has a fat embolism. \nExample Question 2:\nA 55-year-old man comes to the emergency department because of a dry cough and severe chest pain beginning that morning. Two months ago, he was diagnosed with inferior wall myocardial infarction and was treated with stent implantation of the right coronary artery. He has a history of hypertension and hypercholesterolemia. His medications include aspirin, clopidogrel, atorvastatin, and enalapril. His temperature is 38.5Â°C (101.3 °F), pulse is 92/min, respirations are 22/min, and blood pressure is 130/80 mm Hg. Cardiac examination shows a high-pitched scratching sound best heard while sitting upright and during expiration. The remainder of the examination shows no abnormalities. An ECG shows diffuse ST elevations. Serum studies show a troponin I of 0.005 ng/ml (N < 0.01). What is the most likely cause of this patient’s symptoms?\nExample Rationale 2:\n The prior probability of Dressler Syndrome is 0.01%. The patient has diffuse ST elevations, increasing the probability of Dressler Syndrome to 5%. The patient has a scratching murmur which increases the probability to 10%. In the setting of a recent MI the posterior probability of myocardial infarction is 55%. This patient likely has Dressler Syndrome.\n\nQuestion:\n" + str(row["question"])

        reasoning_text = ""
        final_content_text = ""

        try:
            completion = client.chat.completions.create(
                model="openai/gpt-oss-120b",
                messages=[{"role": "user", "content": user_input}],
                temperature=0,
                top_p=1,
                max_tokens=4096,
                stream=True,
                reasoning_effort="high"
            )

            for chunk in completion:
                try:
                    delta = chunk.choices[0].delta

                    reasoning = getattr(delta, "reasoning_content", None)
                    if reasoning is not None:
                        reasoning_text += reasoning

                    content = getattr(delta, "content", None)
                    if content is not None:
                        final_content_text += content

                except Exception:
                    continue

            df.at[idx, "Rationale_29"] = reasoning_text

            if not final_content_text and reasoning_text:
                df.at[idx, "Content_29"] = None
                print("content is empty.")
            else:
                df.at[idx, "Content_29"] = final_content_text

        except Exception as e:
            print(f"Error processing idx={idx}: {e}")
            error_message = f"[ERROR] {e}"
            df.at[idx, "Rationale_29"] = error_message
            df.at[idx, "Content_29"] = error_message

        if (i % SAVE_EVERY == 0) or (i == total):
            df.to_excel(EXCEL_FILE, index=False, engine="openpyxl")
            print(f"Saved progress after {i} / {total}")

    print("All unprocessed rows have been completed.")
